In [1]:
import pandas as pd 


1. 读取数据
2. 分开标签列和特征列
3. 对特征列中类别特征编码
4. 使用全部数据训练模型
5. 输出特征重要性
6. 用auc评价模型效果
7. 用模型给数据打上预测值
8. 根据预测值，在候选用户中，找到实验组用户的相似用户

In [ ]:
# label 1表示实验组用户，0表示候选用户(需要在候选用户里面寻找和实验组用户相似的用户)
y_train = df.pop('label').values
cate_feats=['app_code','mfr','prvn_id','plat']
cate_feats=[ x for x in cate_feats if x in types]
if len(cate_feats)>0:
    cate_encoder = TargetEncoder(cols=cate_feats, min_samples_leaf=3000, smoothing=1)
    dfTrain_cate_tmp = cate_encoder.fit_transform(df[cate_feats], y_train)
    df[cate_feats] = dfTrain_cate_tmp
    del dfTrain_cate_tmp  # 清除引用
    del cate_encoder
feats = df.columns.tolist()
x_train=df.values
del df
gc.collect()
# 训练 PSM 模型
best_parameters = {
    'num_leaves': 63,
    'n_estimators': 100,
    'learning_rate': 0.1
}
global_model = lgb.LGBMClassifier(num_leaves=best_parameters['num_leaves'],
                                      learning_rate=best_parameters['learning_rate'],
                                      objective='binary',
                                      n_estimators=best_parameters['n_estimators'],
                                      random_state=1213, n_jobs=15,
                                      min_child_samples=40, zero_as_missing=False,
                                      min_data_per_group=50, max_cat_threshold=63, cat_smooth=10, silent=False,
                                      verbosity=1, colsample_bytree=0.8, is_unbalance=False)

global_model.fit(x_train, y_train, feature_name=feats,
                 categorical_feature=[])
feat_score = global_model.feature_importances_
feat_imp = [        (feats[k], float(feat_score[k])) for k in range(len(feats))
]
feat_imp = sorted(feat_imp, key=lambda x: x[1], reverse=True)
## 模型输出的特征重要性
print('feat_imp', json.dumps(feat_imp))
print('train finish', flush=True)
## 模型对标签的预测概率
x_train_cls_prob = global_model.predict_proba(x_train)[:, 1].astype(np.float32)
## auc分数，评估模型效果
print('auc score', roc_auc_score(y_train, x_train_cls_prob))
del x_train
gc.collect()

x_train_cls_prob = np.reshape(x_train_cls_prob, (-1))

## 获取训练集和测试集索引
train_idxs=[]
test_idxs=[]
for i in range(x_train_cls_prob.shape[0]):
    if y_train[i]>0.5:
        test_idxs.append(i)
    else:
        train_idxs.append(i)

train_idxs=np.array(train_idxs)
test_idxs=np.array(test_idxs)
## 对训练集样本按照算法预测概率值排序
x_train_prob = x_train_cls_prob[train_idxs]
x_train_prob_sort_idx = np.argsort(x_train_prob)
x_train_prob = x_train_prob[x_train_prob_sort_idx]
train_idxs = train_idxs[x_train_prob_sort_idx]
## 获取测试机的预测概率集合
x_test_prob = x_train_cls_prob[test_idxs]

# 对每个实验组用户，寻找倾向性得分最接近的3个用户 (使用二分搜索加速)
valid_idxs = np.searchsorted(x_train_prob, x_test_prob)

neg=3
sim_uids=[]
for i in range(test_idxs.shape[0]):
    idx_fst = valid_idxs[i]

    if idx_fst >= train_idxs.shape[0]:
        idx_fst = idx_fst - neg

    sim_uids.append((uid[test_idxs[i]], uid[train_idxs[idx_fst:idx_fst + neg]].tolist()))

with open(output_path, 'w') as f:
    for idx, (u1, u2) in enumerate(sim_uids):
        f.write('%s\001%s\001%s\n' % (u1, 1, u1))
        for rnk, t in enumerate(u2):
            if t == u1:
                continue

            f.write('%s\001%s\001%s\n' % (t, 0, u1))

